In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
os.chdir('..')

In [ ]:
import polars as pl
from dotenv import load_dotenv
from pypi_llm.config import Config
from pypi_llm.data.description_cleaner import DescriptionCleaner, CLEANING_FAILED
from pypi_llm.data.reader import DataReader

load_dotenv()
config = Config()

df = DataReader(config.DATA_DIR).read()
df = DescriptionCleaner().clean(df, "description", "description_cleaned")
df = df.filter(~pl.col("description_cleaned").is_null())
df = df.filter(pl.col("description_cleaned")!=CLEANING_FAILED)

In [ ]:
query = "I want to make beautiful data visualizations"

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer(config.EMBEDDINGS_MODEL)
embeddings = model.encode(query)

from pinecone import Pinecone, Index
pc = Pinecone(api_key=config.PINECONE_TOKEN)
index = pc.Index(config.PINECONE_INDEX_NAME)

matches = index.query(
    namespace="ns1",
    vector=embeddings.tolist(),
    top_k=50,
    include_values=False
)

df_matches = pl.from_dicts([{'name' : x['id'], 'similarity': x['score']} for x in matches['matches']])

df_matches = df_matches.join(df, how = 'left', on =  'name')

df_matches.sort('weekly_downloads', descending=True)

